PORCENTAJE DE HOGARES POR DECILES MÁS BAJOS QUE REALIZARON LABORES DE CUIDADO SIN RECIBIR REMUNERACIÓN 

In [ ]:
rm(list=ls())
if(!require('pacman')) install.packages('pacman')
pacman::p_load(tidyverse, srvyr, readr, stringr, dplyr, survey)
#tiempo de espera
options(timeout = 300) 

In [ ]:
#Descarga de archivos
url<-"https://www.inegi.org.mx/contenidos/programas/enigh/nc/2022/microdatos/enigh2022_ns_poblacion_csv.zip"
##Creación de directorio temporal
td<- tempdir()
# Descarga del archivo temporal
tf = tempfile(tmpdir=td, fileext=".zip")
download.file(url, tf)
unzip(tf, files="poblacion.csv", exdir=td, 
      overwrite=TRUE)
fpath=file.path(td,"poblacion.csv")
unlink(td)

In [ ]:
##Se carga tabla población para identificar aquellas personas dedicadas a cuidados
pob <- read_csv(fpath, show_col_types = FALSE) |>
  select(folioviv, foliohog, hor_4, min_4, entidad, numren, parentesco, sexo) |>
  filter(entidad == "09") |>
  mutate(tiempo_cuidados = hor_4 * 60 + min_4) |>
  mutate(cuidados = case_when(
    #tiempo_cuidados >= 1680 ~ 1,
    tiempo_cuidados>=(8*6*60) ~ 1,
    TRUE ~ 0
  )) |>
  filter(parentesco == 101)

In [ ]:
# URL del archivo ZIP para la tabla concentradohogar.csv
url2 <- "https://www.inegi.org.mx/contenidos/programas/enigh/nc/2022/microdatos/enigh2022_ns_concentradohogar_csv.zip"

In [ ]:
# Creación de directorio temporal
td2 <- tempdir()
# Descarga del archivo temporal
tf2 <- tempfile(tmpdir=td2, fileext=".zip")
download.file(url2, tf2)
unzip(tf2, files="concentradohogar.csv", exdir=td2, 
      overwrite=TRUE)
fpath2 <- file.path(td2, "concentradohogar.csv")
unlink(td2)
# Lectura del archivo CSV
concentrado <- read_csv(fpath2, show_col_types = FALSE) |>
  mutate(entidad = substr(folioviv,1,2)) |>
  filter(entidad == "09") |> # Filtramos Ciudad de México
  select(folioviv, foliohog, sexo_jefe, ing_cor, upm, est_dis, factor, est_socio) |>
  left_join(pob, by = c("folioviv", "foliohog")) |> # Se juntaron las bases 
  mutate(ing_mensual = ing_cor / 3,
         decil=ntile(ing_mensual,10))
    

In [ ]:
##Diseño muestral
dm <- concentrado %>%
  as_survey_design(
    ids = upm,
    strata = est_dis,
    weights = factor,
    nest = TRUE)  

In [ ]:
dm%>%
  group_by(est_socio)%>%
  summarise(#ingresop=survey_mean(ing_mensual,vartype="cv"),
            tothog=survey_total(vartype = "cv"),
            hogcuid=survey_total(cuidados,vartype="cv")
             )%>% 
             mutate(indica=hogcuid/tothog*100)%>%
  janitor::adorn_totals()


In [ ]:
concentrado <- read_csv(fpath2, show_col_types = FALSE) |>
  mutate(entidad = substr(folioviv,1,2)) |>
  filter(entidad == "09") |> # Filtramos Ciudad de México
  select(folioviv, foliohog, sexo_jefe, ing_cor, upm, est_dis, factor, est_socio)


In [ ]:
pob <- read_csv(fpath, show_col_types = FALSE) |>
  select(folioviv, foliohog, hor_4, min_4, entidad, numren, parentesco, sexo) |>
  filter(entidad == "09") |>
  mutate(tiempo_cuidados = hor_4 * 60 + min_4) |>
  mutate(cuidadora = case_when(
   # tiempo_cuidados >= 1680 ~ 1,
    tiempo_cuidados>=(8*6*60) ~ 1,
    
    TRUE ~ 0
  ))%>%
  left_join(concentrado, by = c("folioviv", "foliohog"))


In [ ]:
dm <- pob %>%
  as_survey_design(
    ids = upm,
    strata = est_dis,
    weights = factor,
    nest = TRUE) 

In [ ]:
dm%>%
#  group_by(est_socio)%>%
  summarise(pobtot=survey_total(vartype = "cv"),
    personas_cuidadoras=survey_total(cuidadora,vartype="cv"),)